In [1]:
from nltk.stem import WordNetLemmatizer

In [6]:
import nltk
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger") #the dependent text

[nltk_data] Downloading package wordnet to C:\Users\Wei
[nltk_data]     Guo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Wei Guo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [12]:
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [13]:
wordnet_lemmatizer = WordNetLemmatizer()

In [18]:
stopwords = set(w.rstrip() for w in open('stopwords1.txt'))

In [21]:
positive_reviews = BeautifulSoup(open('positive.review', encoding='utf-8' ).read())

D:\anaconda\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\anaconda\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [24]:
positive_reviews = positive_reviews.find_all('review_text')

In [26]:
negative_reviews = BeautifulSoup(open('negative.review', encoding='utf-8' ).read()).find_all('review_text')

D:\anaconda\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\anaconda\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [37]:
def my_token(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

In [38]:
word_index_map = {}
current_index = 0

In [41]:
positive_tokens = []
negative_tokens = []

In [42]:
for review in positive_reviews:
    tokens = my_token(review.text)
    positive_tokens.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [43]:
for review in negative_reviews:
    tokens = my_token(review.text)
    negative_tokens.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [44]:
def token_to_vector(tokens, label):
    x = np.zeros(len(word_index_map)+1)
    for token in tokens:
        i = word_index_map[token]
        x[i] += 1
    x = x/x.sum()
    x[-1] = label
    return x

In [45]:
N = len(positive_tokens) + len(negative_tokens)

In [47]:
import numpy as np

In [49]:
data = np.zeros((N,len(word_index_map)+1))
i = 0

In [50]:
for token in positive_tokens:
    xy = token_to_vector(token,label = 1)
    data[i,:] = xy
    i += 1

In [51]:
for token in negative_tokens:
    xy = token_to_vector(token,label = 0)
    data[i,:] = xy
    i += 1

In [52]:
np.random.shuffle(data)

In [53]:
X = data[:,:-1]
Y = data[:,-1]

In [55]:
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

In [56]:
model = LogisticRegression()
model.fit(Xtrain,Ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [57]:
model.score(Xtest,Ytest)

0.61

In [59]:
len(model.coef_[0])

11424

In [77]:
for word, index in word_index_map.items():
    if model.coef_[0][index] > 0.5:
        print(str(word)+" "+ str(model.coef_[0][index]))

cable 0.5599608359307432
, 0.6210462785202919
've 0.5017304988032769
sound 0.7860976797019568
you 0.8200928040297826
easy 1.393698593737348
quality 1.3313280261152445
perfect 0.7655134448842307
's 0.7041069226306625
fast 0.701575067189443
ha 0.626272075555006
price 2.277876880505877
memory 0.6605290869107718
highly 0.7508120144142116
recommend 0.5489999939866422
little 0.6661481239702465
excellent 1.0023861662762636
love 0.8988762324312071
using 0.515668892932378
speaker 0.7104113048399897
paper 0.5481507950775717
